In [ ]:
import numpy as np
import os
import itertools
import time
from robustness import datasets

import torch
from torch.utils.data import DataLoader
import sys

from qtorch.quant import Quantizer, quantizer
from qtorch.optim import OptimLP
from torch.optim import SGD
from qtorch import FloatingPoint
from qtorch.auto_low import sequential_lower
import math

import itertools
import utils
import train_func as tf

from torchsummary import summary

In [ ]:
# low precision configurations
low_formats = [['Representation_Float', '5', 1],['Representation_Float', '6', 2],
               ['Representation_Float', '7', 3],['Representation_Float', '8', 4],
               ['Representation_Float', '6', 1],['Representation_Float', '7', 2],
               ['Representation_Float', '8', 3],['Representation_Float', '9', 4],
               ['Representation_Float', '7', 1],['Representation_Float', '8', 2],
               ['Representation_Float', '9', 3]]

high_formats = [['Representation_Float', '14', 7],['Representation_Float', '16', 9],
                ['Representation_Float', '18', 11],['Representation_Float', '15', 7],
                ['Representation_Float', '17', 9],['Representation_Float', '19', 11],
                ['Representation_Float', '16', 7],['Representation_Float', '18', 9],
                ['Representation_Float', '20', 11]]

lp_configs = list(itertools.product(low_formats,high_formats))

In [ ]:
mem_matrix = np.zeros((88, 99))

In [ ]:
print("Start Evaluating on CIFAR10")

resolution = (3, 32, 32)
batch_size = 32
arch = 'resnet18'
data = 'cifar10'
epo = 10
bs = 32
lr = 0.001
transform = 'default'
data_dir = './data/'

transforms = tf.load_transforms(transform)
trainset = tf.load_trainset(data, transforms, path=data_dir)
print("Number of classes in {} is: {}".format(data,trainset.num_classes))

for i in range(len(lp_configs)):
    current_config = lp_configs[i]
    print("Currently trains on : {}".format(current_config))
    low_format,high_format = current_config[0],current_config[1]
    man_low,man_high = low_format[2],high_format[2]
    exp_low = int(low_format[1])-man_low-1
    exp_high = int(high_format[1])-man_high-1
    bit_low = FloatingPoint(exp=exp_low, man=man_low)
    bit_high = FloatingPoint(exp=exp_high, man=man_high)
    low_quant_func = lambda: Quantizer(forward_number=bit_low, backward_number=bit_low,
                            forward_rounding="stochastic", backward_rounding="stochastic")
    high_quant = Quantizer(forward_number=bit_high, backward_number=bit_high,
                            forward_rounding="stochastic", backward_rounding="stochastic")
    ## load model
    net = tf.load_architectures(arch, low_quant_func, high_quant, trainset.num_classes)
    mem_matrix[0,i] = summary(net, resolution, batch_size, LP=True)

In [ ]:
print("Start Evaluating on CIFAR100")

resolution = (3, 32, 32)
batch_size = 32
arch = 'resnet18'
data = 'cifar10'
epo = 10
bs = 32
lr = 0.001
transform = 'default'
data_dir = './data/'

for idx in range(20):
    for i in range(len(lp_configs)):
        current_config = lp_configs[i]
        print("Currently trains on : {}".format(current_config))
        low_format,high_format = current_config[0],current_config[1]
        man_low,man_high = low_format[2],high_format[2]
        exp_low = int(low_format[1])-man_low-1
        exp_high = int(high_format[1])-man_high-1
        bit_low = FloatingPoint(exp=exp_low, man=man_low)
        bit_high = FloatingPoint(exp=exp_high, man=man_high)
        low_quant_func = lambda: Quantizer(forward_number=bit_low, backward_number=bit_low,
                                forward_rounding="stochastic", backward_rounding="stochastic")
        high_quant = Quantizer(forward_number=bit_high, backward_number=bit_high,
                                forward_rounding="stochastic", backward_rounding="stochastic")
        ## load model
        net = tf.load_architectures(arch, low_quant_func, high_quant, 5)
        mem_matrix[idx+1,i] = summary(net, resolution, batch_size, LP=True)

In [ ]:
ds_dir = "datasets/my_deca"
resolution = (3, 48, 48)
dataset_list = os.listdir(ds_dir)
for ds_id, ds_name in enumerate(dataset_list):
    test_transforms = tf.load_transforms('deca_test')
    testset = tf.load_trainset(ds_name, test_transforms, train=False, path='datasets/my_deca')
    for i in range(len(lp_configs)):
        current_config = lp_configs[i]
        print("Currently trains on : {}".format(current_config))
        low_format,high_format = current_config[0],current_config[1]
        man_low,man_high = low_format[2],high_format[2]
        exp_low = int(low_format[1])-man_low-1
        exp_high = int(high_format[1])-man_high-1
        bit_low = FloatingPoint(exp=exp_low, man=man_low)
        bit_high = FloatingPoint(exp=exp_high, man=man_high)
        low_quant_func = lambda: Quantizer(forward_number=bit_low, backward_number=bit_low,
                                forward_rounding="stochastic", backward_rounding="stochastic")
        high_quant = Quantizer(forward_number=bit_high, backward_number=bit_high,
                                forward_rounding="stochastic", backward_rounding="stochastic")
        config = "l_{}_{}_h_{}_{}".format(low_format[1],low_format[2],high_format[1],high_format[2])
        ## load model     
        net = tf.load_architectures(arch, low_quant_func, high_quant, testset.num_classes)
        mem_matrix[21+ds_id,i] = summary(net, resolution, batch_size, LP=True)

In [ ]:
print("Start Evaluating on imagenet")
base_model_dir_list = ["saved_models/imagenet_subsets/lp_resnet18+","_epo10_bs32_lr0.001_mom0.9_wd0.0005"]

in_path = 'dir/to/imagenet64x64
in_info_path = 'dir/to/imagenet64x64'

from robustness.tools.imagenet_helpers import ImageNetHierarchy
in_hier = ImageNetHierarchy(in_path,
                            in_info_path)

superclasses_list = []
for cnt, (wnid, ndesc_in, ndesc_total) in enumerate(reversed(in_hier.wnid_sorted)):
    if ndesc_in >= 5:
        #print(f"WordNet ID: {wnid}, Name: {in_hier.wnid_to_name[wnid]}, #ImageNet descendants: {ndesc_in}")
        superclasses_list.append(wnid)
#NEED TO CHANGE
subclass_id_dict = set()
count = 0
used = 0
start = 0
i = 0
superclass_ids = []
class_ranges_list = []
superclass_names = []
#label_map_list = []

while i < 50:
    ancestor_wnid = superclasses_list[count]
    # print(f"Superclass | WordNet ID: {ancestor_wnid}, Name: {in_hier.wnid_to_name[ancestor_wnid]}")
    class_ranges, _ = in_hier.get_subclasses([ancestor_wnid],balanced=True)
    class_ranges_no_dup = set()
    for idx, class_id_imgnet in enumerate(class_ranges[0]):
        if not (class_id_imgnet in subclass_id_dict):
            class_ranges_no_dup.add(class_id_imgnet)
            subclass_id_dict.add(class_id_imgnet)
    label = len(class_ranges_no_dup)
    class_ranges = [{list(class_ranges_no_dup)[i]} for i in range(len(class_ranges_no_dup))]
    if label > 4:
        used += 1
    if label > 4 and used > start:
        superclass_names.append(in_hier.wnid_to_name[ancestor_wnid])
        superclass_ids.append(ancestor_wnid)
        class_ranges_list.append(class_ranges)
        i += 1
    count += 1 

for id in range(50):
    print("Evaluating on Superclass {} with WordNetid {}".format(superclass_names[id],superclass_ids[id]))
    class_ranges = class_ranges_list[id]
    for i in range(len(lp_configs)):
        current_config = lp_configs[i]
        print("Currently trains on : {}".format(current_config))
        low_format,high_format = current_config[0],current_config[1]
        man_low,man_high = low_format[2],high_format[2]
        exp_low = int(low_format[1])-man_low-1
        exp_high = int(high_format[1])-man_high-1
        bit_low = FloatingPoint(exp=exp_low, man=man_low)
        bit_high = FloatingPoint(exp=exp_high, man=man_high)
        low_quant_func = lambda: Quantizer(forward_number=bit_low, backward_number=bit_low,
                                forward_rounding="stochastic", backward_rounding="stochastic")
        high_quant = Quantizer(forward_number=bit_high, backward_number=bit_high,
                                forward_rounding="stochastic", backward_rounding="stochastic")
        config = "l_{}_{}_h_{}_{}".format(low_format[1],low_format[2],high_format[1],high_format[2])
        ## load model
        net = tf.load_architectures(arch, low_quant_func, high_quant, len(class_ranges))
        mem_matrix[38+id,i] = summary(net, resolution, batch_size, LP=True)

In [ ]:
np.save("mem_matrix_full", mem_matrix)